# Install and import necessary library, modules

In [ ]:
from IPython.display import clear_output # To clean up the installing lines
!pip install ultralytics
clear_output()
print("Ultralytics installed!")

In [ ]:
import os
import glob
import yaml
import pandas as pd
from PIL import Image

import torch
from ultralytics import YOLO
device = 0 if torch.cuda.is_available() else 'cpu'

___
# Define the dataset yaml file
YOLO expect the model be trained / fine-tuned on the data by define the yaml file that structure accordingly

We include the training with the dataset of [SignverOD_yolov5_format: Signature Detection](https://www.kaggle.com/datasets/amshaky/signverod-yolov5), which is a modified version of this [SignverOD](https://www.kaggle.com/datasets/victordibia/signverod) dataset. The original dataset has many flaws in it, such as incorrect labeling, duplicates and misalignments. Moreover, the modified version is also prepared with the feasible YOLO-compatible structure.

The dataset introduces 4 objects to detach, which are all version of handwritten:
- Signatures
- Redactions
- Initials
- Dates

All of the fours are handwritten, which can easily mislead the modify into wrong localization.

In [ ]:
def config_yaml(output_file: str = '/kaggle/working/signature.yaml'):
    '''
    Write the yaml file to define the dataset following YOLO's structure

    Args:
        output_file (str): The location to save the file.
    '''
    yaml_content = {
        'train': '/kaggle/input/signverod-yolov5/signs_yolo_format/signs_yolo_format/images/train',
        'val': '/kaggle/input/signverod-yolov5/signs_yolo_format/signs_yolo_format/images/valid',
        'nc': 4,
        'names': ['signature', 'initial', 'redaction', 'date'],
    }
    with open(output_file, 'w') as f:
        yaml.dump(yaml_content, f, default_flow_style=False)
        print(f"Save to {output_file}")

config_yaml()

___
# Load pre-trained model from the Ultralytics and perform train on the signature dataset

In [ ]:
model = YOLO("yolo11s.pt")

In [ ]:
%%capture train_out # To limit the output printed, uncomment if needed
num_epochs = 50
rs = model.train(
    data='/kaggle/working/signature.yaml', # Dataset yaml file
    epochs=num_epochs, # Number of epochs
    imgsz=768, # The size of the images to be resized, can be up-sized for such small object detections like signatures
    batch=16, 
    device=device, # Using GPU or CPU
    project='/kaggle/working/yolo_detect', # Naming the folder to save the training data
    name='exp',
    save=True,
    pretrained=False,
)

# After training, the model will output 2 file within the defined_folder/weights:
# - best.pt
# - last.pt

___
# Push the trained model to hub

In [ ]:
# from huggingface_hub import HfApi
# HF_TOKEN = "YOUR-HF-TOKEN"

# api = HfApi(token=HF_TOKEN)
# api.upload_folder(
#     folder_path="path-to-best.pt", # Change this path to the best.pt training file, or the earlier `model_path`
#     repo_id="Mels22/Signature-Detection-Verification",
#     repo_type="model",
# )

___
# Evaluate the trained model

The model is pulled back from Hugging Face

In [ ]:
from huggingface_hub import hf_hub_download
MODEL_REPO = "Mels22/Signature-Detection-Verification"

# If the file is not in the current dir, download it from hub
detector_filename = "detector_yolo_4cls.pt"
if not os.path.isfile(detector_filename):
    print(f"Download {detector_filename} from hub")
    hf_hub_download(repo_id=MODEL_REPO, filename=detector_filename, local_dir='.')

In [ ]:
%%capture val_out
model = YOLO(detector_filename) # Load the model and begin eval on it
metrics = model.val(
    data='signature.yaml', # Dataset yaml 
)

In [ ]:
dat = {
    'precision': metrics.box.p,
    'recall': metrics.box.r,
    'mAP@50': metrics.box.ap50,
    'mAP50-95': metrics.box.ap,
} # Take out the metrics of the detections

df = pd.DataFrame(dat, index=['signature', 'initial', 'redaction', 'date'])
df = pd.concat([df, pd.DataFrame([metrics.mean_results()], index=['mean'], columns=df.columns)]) # Add a average row

pd.DataFrame(df) # Load out the data on evaluation

___
# Inference with the model

In [ ]:
def get_predictions(model, image_path, show=False):
    """
    Get bounding boxes and confidences from a single image.
    
    Args:
        model: Loaded YOLO model
        image_path (str): Path to an image
        
    Returns:
        List of dicts with 'label', 'confidence', and 'bbox' keys
    """
    results = model(image_path)[0]
    if show:
        results[0].show() # Display the detected image
    preds = []
    for box, cls, conf in zip(results.boxes.xywhn, results.boxes.cls, results.boxes.conf):
        x1, y1, x2, y2 = map(float, box.tolist())
        preds.append({
            'label': model.names[int(cls)],
            'confidence': float(conf),
            'bbox': (x1, y1, x2, y2)
        }) # Loop through each detection and note down into the predictions
    return preds

def run_inference(model_path, input_path, show=False):
    """
    Load model and run inference on a single image or a folder of images.
    
    Args:
        model_path (str): Path to YOLO model (.pt)
        input_path (str): Path to a single image or a folder
        
    Returns:
        Dictionary mapping image filenames to lists of predictions
    """
    # Load the model and find the relevant image file/folder
    model = YOLO(model_path)
    if os.path.isdir(input_path):
        image_paths = glob.glob(os.path.join(input_path, '*.jpg')) + glob.glob(os.path.join(input_path, '*.png'))
    else:
        image_paths = [input_path]

    # Get all predictions
    all_results = {}
    for img_path in image_paths:
        preds = get_predictions(model, img_path, show)
        all_results[os.path.basename(img_path)] = preds
    
    return all_results

In [ ]:
run_inference(
    model_path=detector_filename, 
    input_path='/kaggle/input/signverod-yolov5/signs_yolo_format/signs_yolo_format/images/valid/2030.png', # Test image path (file/folder)
    show=True,
)